In [2]:
!pip install unsloth vllm
!pip install --upgrade pillow
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.0 MB/s eta 0:00:0000:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 4.0.10
    Uninstalling widgetsnbextension-4.0.10:
      Successfully uninstalled widgetsnbextension-4.0.10
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.10
    Uninstalling jupyterlab_widgets-3.0.10:
      Successfully uninstalled jupyterlab_widgets-3.0.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 8.1.2
    Uninstalling ipywidgets-8.1.2:
      Successfully uninstalled ipywidgets-8.1.2


In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

In [10]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
    device = "cuda:1"
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-bnb-4bit with actual GPU utilization = 10.83%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.11 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.24 GB. Also swap space = 6 GB.
INFO 03-01 15:38:31 config.py:549] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwar

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-01 15:38:35 model_runner.py:1115] Loading model weights took 5.3229 GB
INFO 03-01 15:38:36 worker.py:267] Memory profiling takes 0.62 seconds
INFO 03-01 15:38:36 worker.py:267] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.11) = 8.57GiB
INFO 03-01 15:38:36 worker.py:267] model weights take 5.32GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.74GiB; the rest of the memory reserved for KV Cache is 2.51GiB.
INFO 03-01 15:38:36 executor_base.py:111] # cuda blocks: 1284, # CPU blocks: 3072
INFO 03-01 15:38:36 executor_base.py:116] Maximum concurrency for 1024 tokens per request: 20.06x
INFO 03-01 15:38:36 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:18<00:00,  1.25it/s]

INFO 03-01 15:38:55 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.22 GiB
INFO 03-01 15:38:55 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 19.76 seconds


In [11]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
import re
import random

# Load and prep dataset
SYSTEM_PROMPT = """
You are a poker math calculator. Given a poker situation, respond in the following format:
<reasoning>
Calculate step by step: Equity Odds, Pot Odds, Expected Value (EV), and Minimum Defense Frequency (MDF).
</reasoning>
<answer>
Choose one of: fold, check, call, bet, raise
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

import random

# Equity Odds Reward Function
def equity_odds_reward_func(your_hand, community_cards, action, **kwargs) -> float:
    """
    Rewards decisions based on equity advantage/disadvantage and chosen action.
    
    Args:
        your_hand: List of your hole cards
        community_cards: List of community cards
        action: Player's chosen action (fold, check, call, bet, raise, allin)
        
    Returns:
        Float representing reward based on equity quality and action appropriateness
    """
    equity = calculate_simplified_equity(your_hand, community_cards)
    
    # Evaluate action based on equity
    if equity >= 0.7:  # Very strong hand
        if action in ["bet", "raise", "allin"]:
            return 1.0  # Aggressive with strong hands is optimal
        elif action in ["call", "check"]:
            return 0.6  # Passive with strong hands loses value
        else:  # fold
            return 0.0  # Folding strong hands is terrible
    elif equity >= 0.5:  # Above average hand
        if action in ["bet", "raise"]:
            return 0.8  # Aggression is good
        elif action in ["call", "check"]:
            return 0.7  # Calling/checking is reasonable
        elif action == "allin":
            return 0.5  # Overcommitting with decent hands
        else:  # fold
            return 0.1  # Folding good hands is suboptimal
    elif equity >= 0.4:  # Slightly above average
        if action in ["bet", "call"]:
            return 0.7  # Reasonable plays
        elif action == "check":
            return 0.6  # Checking is fine
        elif action == "raise":
            return 0.5  # Raising is slightly aggressive
        elif action == "allin":
            return 0.3  # Overcommitting
        else:  # fold
            return 0.4  # Folding might be reasonable depending on context
    elif equity >= 0.3:  # Below average but playable
        if action in ["call", "check"]:
            return 0.6  # Reasonable with marginal hands
        elif action == "bet":
            return 0.5  # Can be a bluff or thin value
        elif action == "fold":
            return 0.5  # Folding marginal hands can be correct
        elif action == "raise":
            return 0.4  # Raising as a bluff can work
        else:  # allin
            return 0.1  # Overcommitting with weak hands
    else:  # Weak hand
        if action == "fold":
            return 0.9  # Folding weak hands is usually correct
        elif action == "check":
            return 0.6  # Checking to see free cards
        elif action in ["bet", "raise"]:
            return 0.4  # Bluffing can be strategic
        elif action == "call":
            return 0.2  # Calling with weak hands is usually bad
        else:  # allin
            return 0.1  # All-in with weak hands is rarely optimal



In [ ]:
# Pot Odds Reward Function
def pot_odds_reward_func(pot_size, amount_to_call, your_hand, community_cards, action, **kwargs) -> float:
    """
    Rewards decisions based on pot odds vs equity comparison and chosen action.
    
    Args:
        pot_size: Current size of the pot
        amount_to_call: Amount needed to call
        your_hand: Your hole cards for equity calculation
        community_cards: Community cards for equity calculation
        action: Player's chosen action
        
    Returns:
        Float representing reward based on pot odds decision quality
    """
    if amount_to_call == 0:
        # No pot odds calculation needed if no call required
        return 0.5
        
    pot_odds = amount_to_call / (pot_size + amount_to_call)
    equity = calculate_simplified_equity(your_hand, community_cards)
    
    # Evaluate action based on pot odds vs equity
    if equity > pot_odds:  # +EV call
        margin = equity - pot_odds
        
        if action in ["call", "raise", "allin"]:
            # Reward for making +EV calls/raises
            if margin > 0.2:  # Extremely +EV
                return 1.0
            elif margin > 0.1:  # Very +EV
                return 0.9
            else:  # Slightly +EV
                return 0.8
        elif action == "check" and amount_to_call == 0:
            return 0.7  # Checking when no call required is fine
        elif action == "bet" and amount_to_call == 0:
            return 0.8  # Betting when no call required with +EV hand
        else:  # fold
            return 0.1  # Folding when calling is +EV is bad
    else:  # -EV call
        margin = pot_odds - equity
        
        if action == "fold":
            # Reward for folding -EV calls
            if margin > 0.2:  # Extremely -EV
                return 0.9
            elif margin > 0.1:  # Very -EV
                return 0.8
            else:  # Slightly -EV
                return 0.7
        elif action == "call":
            # Penalize -EV calls
            if margin > 0.2:  # Extremely -EV
                return 0.1
            elif margin > 0.1:  # Very -EV
                return 0.2
            else:  # Slightly -EV
                return 0.3
        elif action in ["raise", "allin"]:
            # Heavily penalize raising with -EV hands
            return 0.1
        elif action == "check" and amount_to_call == 0:
            return 0.6  # Checking when no call required is fine
        else:  # bet
            return 0.3  # Betting with -EV hand (could be a bluff)
        
# Expected Value Reward Function
def ev_reward_func(equity, pot_size, amount_to_call, action, amount_you_have_raised=0, **kwargs) -> float:
    """
    Rewards decisions based on expected value calculation and chosen action.
    
    Args:
        equity: Your equity in the hand
        pot_size: Current size of the pot
        amount_to_call: Amount needed to call
        action: Player's chosen action
        amount_you_have_raised: Amount you've already invested
        
    Returns:
        Float representing reward based on EV quality and action
    """
    # Calculate EV of calling
    ev_call = (equity * pot_size) - ((1 - equity) * amount_to_call)
    
    # Normalize EV relative to pot size for consistent scaling
    normalized_ev = ev_call / pot_size if pot_size > 0 else 0
    
    # Evaluate action based on EV
    if normalized_ev > 0.2:  # Strongly +EV
        if action in ["call", "raise", "allin"]:
            return 1.0  # Correct to continue with strongly +EV
        elif action == "check" and amount_to_call == 0:
            return 0.7  # Checking is fine but less optimal
        elif action == "bet":
            return 0.9  # Betting is good with +EV
        else:  # fold
            return 0.1  # Folding +EV is bad
    elif normalized_ev > 0:  # Slightly +EV
        if action in ["call", "bet"]:
            return 0.8  # Good to continue with +EV
        elif action == "raise":
            return 0.7  # Raising can be good but risky
        elif action == "check" and amount_to_call == 0:
            return 0.6  # Checking is reasonable
        elif action == "allin":
            return 0.5  # Overcommitting with marginal +EV
        else:  # fold
            return 0.3  # Folding slight +EV is suboptimal
    elif normalized_ev > -0.1:  # Slightly -EV
        if action == "fold":
            return 0.7  # Folding slight -EV is reasonable
        elif action == "call":
            return 0.4  # Calling slight -EV can be justified
        elif action == "check" and amount_to_call == 0:
            return 0.6  # Checking is fine
        elif action == "bet":
            return 0.5  # Betting as bluff can be strategic
        elif action == "raise":
            return 0.3  # Raising -EV is risky
        else:  # allin
            return 0.1  # All-in with -EV is bad
    else:  # Strongly -EV
        if action == "fold":
            return 0.9  # Folding strongly -EV is correct
        elif action == "check" and amount_to_call == 0:
            return 0.6  # Checking when free is fine
        elif action == "bet":
            return 0.4  # Betting as bluff can sometimes work
        elif action == "call":
            return 0.2  # Calling strongly -EV is bad
        else:  # raise, allin
            return 0.1  # Raising or going all-in with -EV is terrible
        


In [ ]:
# Minimum Defense Frequency Reward Function
def mdf_reward_func(bet_size, pot_size, your_position, action, action_previous_players, **kwargs) -> float:
    """
    Rewards decisions based on MDF principles and chosen action.
    
    Args:
        bet_size: Size of the bet you're facing
        pot_size: Size of the pot before the bet
        your_position: Your position at the table
        action: Player's chosen action
        action_previous_players: Actions taken by players before you
        
    Returns:
        Float representing reward based on MDF-based decision quality
    """
    # Calculate optimal defense frequency
    mdf = pot_size / (pot_size + bet_size) if bet_size > 0 else 0.5
    
    # Determine if this is a spot where MDF is applicable
    is_in_position = is_position_favorable(your_position, action_previous_players)
    is_facing_bet = "bet" in action_previous_players or "raise" in action_previous_players
    
    if not is_facing_bet:
        # MDF doesn't apply when not facing a bet
        return 0.5
    
    # Evaluate action based on MDF principles
    if action in ["call", "raise", "allin"]:  # Defending
        if is_in_position:
            # In position, defending according to MDF is important
            return 0.8 + (mdf * 0.2)  # Higher reward for defending in position
        else:
            # Out of position, defending is still important but slightly less optimal
            return 0.7 + (mdf * 0.2)
    elif action == "fold":  # Not defending
        if mdf > 0.7:  # Should defend most hands
            return 0.3  # Penalize folding when should defend often
        elif mdf > 0.5:
            return 0.5  # Moderate penalty for folding
        else:  # Can fold most hands
            return 0.7  # Folding is often correct when MDF is low
    else:  # check, bet - not directly applicable to MDF
        return 0.5


In [ ]:
# Stack-to-Pot Ratio Reward Function
def spr_reward_func(pot_size, your_chips, opponent_chips, action, **kwargs) -> float:
    """
    Rewards decisions based on stack-to-pot ratio considerations and chosen action.
    
    Args:
        pot_size: Current size of the pot
        your_chips: Amount of chips you have
        opponent_chips: Amount of chips opponent has
        action: Player's chosen action
        
    Returns:
        Float representing reward based on SPR decision quality
    """
    # Calculate effective stack (minimum of your stack and opponent's stack)
    effective_stack = min(your_chips, opponent_chips)
    
    # Calculate SPR
    spr = effective_stack / pot_size if pot_size > 0 else 10
    
    # Evaluate action based on SPR
    if spr < 1:  # Very committed - pot committed situations
        if action in ["call", "raise", "allin"]:
            return 1.0  # Correct to continue when pot committed
        else:  # fold, check, bet
            return 0.2  # Incorrect to fold when pot committed
    elif spr < 3:  # Moderately committed
        if action in ["call", "bet"]:
            return 0.8  # Good to continue
        elif action == "raise":
            return 0.7  # Raising can be good
        elif action == "allin":
            return 0.6  # All-in can be justified
        elif action == "check":
            return 0.5  # Checking is passive
        else:  # fold
            return 0.3  # Folding is often incorrect with low SPR
    elif spr < 7:  # Standard SPR
        if action in ["bet", "raise"]:
            return 0.7  # Aggression is good
        elif action in ["call", "check"]:
            return 0.6  # Passive play is acceptable
        elif action == "fold":
            return 0.5  # Folding can be correct
        else:  # allin
            return 0.4  # All-in is often an overcommitment
    else:  # Very deep stacked
        if action in ["bet", "raise"]:
            return 0.7  # Controlled aggression is good
        elif action in ["call", "check"]:
            return 0.6  # Passive play is fine
        elif action == "fold":
            return 0.6  # Folding is often correct
        else:  # allin
            return 0.2  # All-in is usually an overcommitment with deep stacks

"""
# Emotional Read Reward Function
def emotional_read_reward_func(emotions_other_players, your_hand, community_cards, action, **kwargs) -> float:

    Rewards decisions based on reading opponents' emotions and chosen action.
    
    Args:
        emotions_other_players: Dictionary of player emotions
        your_hand: Your hole cards
        community_cards: Community cards
        action: Player's chosen action
        
    Returns:
        Float representing reward based on emotional exploitation
  
    # Extract relevant emotions (simplified)
    has_nervous_opponent = "nervous" in emotions_other_players.values()
    has_confident_opponent = "confident" in emotions_other_players.values()
    has_tilted_opponent = "tilted" in emotions_other_players.values()
    
    # Calculate hand strength
    equity = calculate_simplified_equity(your_hand, community_cards)
    
    # Evaluate action based on emotional reads
    if has_nervous_opponent:
        if equity > 0.4:  # Decent hand vs nervous opponent
            if action in ["bet", "raise"]:
                return 0.9  # Aggressive with value hands vs nervous opponents
            elif action in ["call", "check"]:
                return 0.6  # Passive play misses value
            elif action == "allin":
                return 0.7  # Can scare nervous opponent
            else:  # fold
                return 0.1  # Folding value vs nervous opponent is bad
        else:  # Weak hand vs nervous opponent
            if action in ["bet", "raise"]:
                return 0.7  # Bluffing nervous opponents can work
            elif action == "check":
                return 0.6  # Checking to see free cards
            elif action == "fold":
                return 0.5  # Folding weak hands is reasonable
            else:  # call, allin
                return 0.3  # Committing with weak hands is risky
    
    elif has_confident_opponent:
        if equity < 0.3:  # Weak hand vs confident opponent
            if action == "fold":
                return 0.8  # Folding weak hands vs confident opponents is good
            elif action == "check" and "check" in action_previous_players:
                return 0.6  # Checking behind is okay
            elif action in ["call", "bet", "raise", "allin"]:
                return 0.2  # Committing against confident opponent with weak hand is bad
        else:  # Strong hand vs confident opponent
            if action in ["call", "raise"]:
                return 0.8  # Let confident opponent bet into you
            elif action == "check":
                return 0.7  # Checking to induce bets
            elif action == "bet":
                return 0.6  # Betting might make opponent fold
            elif action == "allin":
                return 0.5  # Overcommitting might scare opponent
            else:  # fold
                return 0.1  # Folding good hands is bad
    
    elif has_tilted_opponent:
        if equity > 0.3:  # Decent hand vs tilted opponent
            if action in ["bet", "raise"]:
                return 0.9  # Aggressive vs tilted opponents is optimal
            elif action in ["call", "check"]:
                return 0.6  # Passive play misses value
            elif action == "allin":
                return 0.7  # Can get called by tilted opponent
            else:  # fold
                return 0.1  # Folding vs tilted opponent is bad
        else:  # Weak hand vs tilted opponent
            if action in ["bet", "raise"]:
                return 0.7  # Bluffing can work against tilted opponents
            elif action == "check":
                return 0.5  # Checking is safe
            elif action == "fold":
                return 0.6  # Folding weak hands is reasonable
            elif action == "call":
                return 0.4  # Calling with weak hands can be exploitative
            else:  # allin
                return 0.3  # Overcommitting is risky
    
    else:  # No clear emotional reads
        return 0.5  # Neutral reward when no emotional information
    """


# Combined Poker Decision Reward Function
def poker_decision_reward_func(your_hand, community_cards, pot_size, amount_to_call, 
                              your_position, your_chips, opponent_chips, action,
                           emotions_other_players, action_previous_players, **kwargs) -> float:

    """
    Args:
        your_hand: Your hole cards
        community_cards: Community cards
        pot_size: Current pot size
        amount_to_call: Amount needed to call
        your_position: Your position at the table
        your_chips: Your chip stack
        opponent_chips: Opponent's chip stack
        action: Player's chosen action (fold, check, call, bet, raise, allin)
        emotions_other_players: Dictionary of player emotions
        action_previous_players: Actions of previous players
        
    Returns:
        Float representing overall decision quality
    """
    # Calculate individual rewards
    equity = calculate_simplified_equity(your_hand, community_cards)
    equity_reward = equity_odds_reward_func(your_hand, community_cards, action)
    pot_odds_reward = pot_odds_reward_func(pot_size, amount_to_call, your_hand, community_cards, action)
    ev_reward = ev_reward_func(equity, pot_size, amount_to_call, action)
    mdf_reward = mdf_reward_func(amount_to_call, pot_size, your_position, action, action_previous_players)
    spr_reward = spr_reward_func(pot_size, your_chips, opponent_chips, action)
    emotional_reward = emotional_read_reward_func(emotions_other_players, your_hand, community_cards, action)
    
    # Weight the rewards based on their importance in poker decision making
    # Adjust weights based on the specific situation
    is_facing_bet = "bet" in action_previous_players or "raise" in action_previous_players
    
    if is_facing_bet:
        # When facing a bet, pot odds and EV are more important
        weighted_reward = (
            equity_reward * 0.20 +
            pot_odds_reward * 0.30 +
            ev_reward * 0.25 +
            mdf_reward * 0.10 +
            spr_reward * 0.10 +
            emotional_reward * 0.05
        )
    else:
        # When not facing a bet, equity and position play larger roles
        weighted_reward = (
            equity_reward * 0.30 +
            pot_odds_reward * 0.15 +
            ev_reward * 0.20 +
            mdf_reward * 0.05 +
            spr_reward * 0.15 +
            emotional_reward * 0.15
        )
    
    # Apply action-specific adjustments
    if action == "fold" and equity > 0.7:
        # Heavily penalize folding very strong hands
        weighted_reward *= 0.3
    elif action == "allin" and equity < 0.3 and spr > 5:
        # Penalize going all-in with weak hands when deep stacked
        weighted_reward *= 0.5
    elif action in ["bet", "raise"] and "tilted" in emotions_other_players.values():
        # Bonus for aggressive actions against tilted players
        weighted_reward *= 1.2
        weighted_reward = min(1.0, weighted_reward)  # Cap at 1.0
    
    return weighted_reward

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        equity_odds_reward_func,
        pot_odds_reward_func,
        ev_reward_func,
        mdf_reward_func,
        spr_reward_func,
        poker_decision_reward_func
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()